In [1]:
import os, sys
import toml
import pandas as pd 
import datetime
# Get the absolute path of the project root
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../utils")))

####
# 2/11/2025
# This script is used to calculate rec bike share over all bike trips in NHTS 2017 data.
# The rec bike share is calculated as the number of bike trips that are made in good weather months (April to October) over all bike trips.

#####
# Configure the column names of the survey data
#####
# os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))
config = toml.load(os.path.join(os.getcwd(), '../config.toml'))

trips_file = r'I:\Modeling and Analysis Group\03_Data\Surveys\NHTS_2017\trippub.csv'
BASE_PATH = config['BASE_PATH']
YEAR = ['2016', '2017']
HOUSEHOLD_SIZE_COLUMN = config['HOUSEHOLD_SIZE_COLUMN']
TRIP_WEIGHT_COLUMN = config['TRIP_WEIGHT_COLUMN']
TRIP_DEST_PURP = config['TRIP_DEST_PURP']
TRIP_ORIG_PURP = config['TRIP_ORIGIN_PURP']
SELECTED_TRIPS_COLS = config['SELECTED_TRIP_COLS']
GOODMONTH_FOR_BIKE = config['GOODMONTH_FOR_BIKE']
DISTANCE_COLUMN = config['DISTANCE_COLUMN']
JOURNEY_TIME_COLUMN = config['JOURNEY_TIME_COLUMN']
MODE_CATEGORY_COLUMN = config['MODE_CATEGORY_COLUMN']
NHTS_PURP_DICT = config['NHTS_PURP_DICT']
NHTS_PURP_DICT = {int(k): v for k, v in NHTS_PURP_DICT.items()}
TRIP_PURP_4STEP = config['TRIP_PURP_4STEP']

output_location = os.path.join(os.path.join(BASE_PATH, 'processed'))

trips_df = pd.read_csv(trips_file)

In [2]:
def day_to_number(day_name):
    day_mapping = {"Sunday": 1, "Monday": 2, "Tuesday": 3, "Wednesday": 4,
        "Thursday": 5, "Friday": 6, "Saturday": 7}
    if day_name in day_mapping.keys():
        return day_mapping[day_name]
    else:
        return 0

## Total Bike Trips by Purpose

In [3]:
goodmonth = [int(f"{year}{month}") for year in YEAR for month in GOODMONTH_FOR_BIKE]
# bike mode, travelled in April through November
bike_trips_df = trips_df.loc[(trips_df[MODE_CATEGORY_COLUMN] == 2) & (trips_df['TDAYDATE'].isin(goodmonth)) & (trips_df[DISTANCE_COLUMN] <= 200) & (trips_df[DISTANCE_COLUMN] > 0), SELECTED_TRIPS_COLS].copy()
BMT_COL = 'BMT'
BHT_COL = 'BHT'
# if the trip is a loop trip, count it as two trips
bike_trips_df['Num_Trips'] = 1
bike_trips_df.loc[bike_trips_df['LOOP_TRIP'] == 1, 'Num_Trips'] = 2
bike_trips_df[TRIP_WEIGHT_COLUMN] = bike_trips_df[TRIP_WEIGHT_COLUMN] * bike_trips_df['Num_Trips']
bike_trips_df[BMT_COL] = bike_trips_df[DISTANCE_COLUMN] * bike_trips_df[TRIP_WEIGHT_COLUMN]
bike_trips_df[BHT_COL] = bike_trips_df[JOURNEY_TIME_COLUMN] * bike_trips_df[TRIP_WEIGHT_COLUMN]

weekdays = [day_to_number(day) for day in config['WEEKDAYS']]
weekday_bike_trips = bike_trips_df.loc[bike_trips_df['TRAVDAY'].isin(weekdays)]

# weekday_bike_trips is the unvierse of bike trips that are used to calculate rec bike
weekday_bike_trips.to_csv(os.path.join(output_location, 'weekday_bike_trips.csv'), index=False)
print('Bike trips exported to weekday_bike_trips.csv')

Bike trips exported to weekday_bike_trips.csv


In [4]:
agg_method = {DISTANCE_COLUMN: 'sum', JOURNEY_TIME_COLUMN: 'sum', TRIP_WEIGHT_COLUMN: 'sum', BMT_COL: 'sum',BHT_COL: 'sum', TRIP_DEST_PURP: 'count', 'Num_Trips':'sum'}
bikes_by_purp = weekday_bike_trips[[TRIP_DEST_PURP, DISTANCE_COLUMN, JOURNEY_TIME_COLUMN, BMT_COL, BHT_COL, TRIP_WEIGHT_COLUMN, 'Num_Trips']].groupby(TRIP_DEST_PURP).agg(agg_method)
total_weighted_bike_trips = weekday_bike_trips[TRIP_WEIGHT_COLUMN].sum()
bikes_by_purp['Share'] = bikes_by_purp[TRIP_WEIGHT_COLUMN] / total_weighted_bike_trips
DIST_COL_AVG = DISTANCE_COLUMN + '_unweighted_avg'
TIME_COL_AVG = JOURNEY_TIME_COLUMN + '_unweighted_avg'
DIST_COL_WEIGHTED_AVG = DISTANCE_COLUMN + '_weighted_avg'
TIME_COL_WEIGHTED_AVG = JOURNEY_TIME_COLUMN + '_weighted_avg'

# Num_Record is the number of records (rows) for each purpose
bikes_by_purp.rename(columns={TRIP_DEST_PURP:'Num_Record'}, inplace=True)
bikes_by_purp[DIST_COL_AVG] = bikes_by_purp[DISTANCE_COLUMN] / bikes_by_purp['Num_Record'] 
bikes_by_purp[TIME_COL_AVG] = bikes_by_purp[JOURNEY_TIME_COLUMN] / bikes_by_purp['Num_Record']
bikes_by_purp[DIST_COL_WEIGHTED_AVG] = bikes_by_purp[BMT_COL] / bikes_by_purp[TRIP_WEIGHT_COLUMN]
bikes_by_purp[TIME_COL_WEIGHTED_AVG] = bikes_by_purp[BHT_COL] / bikes_by_purp[TRIP_WEIGHT_COLUMN]

bikes_by_purp['PURP'] = bikes_by_purp.index.map(NHTS_PURP_DICT)
bikes_by_purp = bikes_by_purp[['PURP'] + list(bikes_by_purp.columns.difference(['PURP']))]
# total bike trips (counts) and weighted trips
bikes_weighted_total = bikes_by_purp[TRIP_WEIGHT_COLUMN].sum()
bikes_unweighted_total = bikes_by_purp['Num_Trips'].sum()
print(f'Total bike trips (weighted): {bikes_weighted_total}, Total bike trips (unweighted): {bikes_unweighted_total}')



Total bike trips (weighted): 1631458932.8908012, Total bike trips (unweighted): 3992


In [5]:
# recbike: purpose 1 and 16, loop trip.
rec_bike_trips_df = weekday_bike_trips.loc[(weekday_bike_trips[TRIP_DEST_PURP].isin([1,16])) & (weekday_bike_trips[TRIP_ORIG_PURP].isin([1,16])) & (weekday_bike_trips['LOOP_TRIP']==1)]

# calculate the share of recbike trips by TRIP_PURP_4STEP and TRIP_DEST_PURP.
# TRIP_PURP_4STEP is the 4-step purpose of the trip destination. Home based recbike trips start and end at home. None home based recbike
# trips presumably start and end at parks.

rec_bikes_by_purp = rec_bike_trips_df[[TRIP_DEST_PURP, TRIP_PURP_4STEP, DISTANCE_COLUMN, JOURNEY_TIME_COLUMN, BMT_COL, BHT_COL, TRIP_WEIGHT_COLUMN, 'Num_Trips']].groupby([TRIP_DEST_PURP, TRIP_PURP_4STEP]).agg(agg_method).copy()
total_rec_weighted_bike_trips = rec_bikes_by_purp[TRIP_WEIGHT_COLUMN].sum()
rec_bikes_by_purp['Weighted Share of RecBike'] = rec_bikes_by_purp[TRIP_WEIGHT_COLUMN] / total_rec_weighted_bike_trips
rec_bikes_by_purp['Weighted Share of Allbike'] = rec_bikes_by_purp[TRIP_WEIGHT_COLUMN] / total_weighted_bike_trips


rec_bikes_by_purp.rename(columns={TRIP_DEST_PURP:'Num_Record'}, inplace=True)
total_rec_unweighted_bike_trips = rec_bikes_by_purp['Num_Trips'].sum()
rec_bikes_by_purp['Unweighted Share of RecBike'] = rec_bikes_by_purp['Num_Trips'] / total_rec_unweighted_bike_trips
rec_bikes_by_purp['Unweighted Share of Allbike'] = rec_bikes_by_purp['Num_Trips'] / bikes_unweighted_total


rec_bikes_by_purp[DIST_COL_AVG] = rec_bikes_by_purp[DISTANCE_COLUMN] / rec_bikes_by_purp['Num_Record'] 
rec_bikes_by_purp[TIME_COL_AVG] = rec_bikes_by_purp[JOURNEY_TIME_COLUMN] / rec_bikes_by_purp['Num_Record']
rec_bikes_by_purp[DIST_COL_WEIGHTED_AVG] = rec_bikes_by_purp[BMT_COL] / rec_bikes_by_purp[TRIP_WEIGHT_COLUMN]
rec_bikes_by_purp[TIME_COL_WEIGHTED_AVG] = rec_bikes_by_purp[BHT_COL] / rec_bikes_by_purp[TRIP_WEIGHT_COLUMN]
rec_bikes_by_purp['PURP'] = rec_bikes_by_purp.index.get_level_values(0).map(NHTS_PURP_DICT)
rec_bikes_by_purp = rec_bikes_by_purp[['PURP'] + list(rec_bikes_by_purp.columns.difference(['PURP']))]


# total rec bike trips (counts) and weighted trips
recb_weighted_total = rec_bikes_by_purp[TRIP_WEIGHT_COLUMN].sum()
recb_unweighted_total = rec_bikes_by_purp['Num_Trips'].sum()
print(f'Total rec bike trips (weighted): {recb_weighted_total}, Total rec bike trips (unweighted): {recb_unweighted_total}')

rec_bike_weighted_share = recb_weighted_total / bikes_weighted_total
rec_bike_unweighted_share = recb_unweighted_total / bikes_unweighted_total

print(f'Recreational bike trips share (weighted): {rec_bike_weighted_share}, Recreational bike trips share (unweighted): {rec_bike_unweighted_share}')

rec_bikes_by_purp

Total rec bike trips (weighted): 114159555.9130231, Total rec bike trips (unweighted): 478
Recreational bike trips share (weighted): 0.06997390716463972, Recreational bike trips share (unweighted): 0.11973947895791583


PURP  \
WHYTO TRIPPURP                                                      
1     HBO                                                    Home   
      HBSOCREC                                               Home   
16    HBSOCREC  Exercise (go for a jog, walk, walk the dog, go...   
      NHB       Exercise (go for a jog, walk, walk the dog, go...   

                         BHT           BMT  Num_Record  Num_Trips  TRPMILES  \
WHYTO TRIPPURP                                                                
1     HBO       5.154861e+09  6.242386e+08         175        350  1431.553   
      HBSOCREC  1.142163e+08  4.392255e+07          18         36   113.304   
16    HBSOCREC  5.911484e+08  1.029433e+08          29         58   202.363   
      NHB       1.896660e+08  3.699670e+07          17         34   155.600   

                TRPMILES_unweighted_avg  TRPMILES_weighted_avg  TRVLCMIN  \
WHYTO TRIPPURP                                                             
1     HBO                      8.180303               6.979597     10224   
      HBSOCREC                 6.294667               6.115809       675   
16    HBSOCREC                 6.978034               8.044315      1229   
      NHB                      9.152941               7.800117       837   

                TRVLCMIN_unweighted_avg  TRVLCMIN_weighted_avg  \
WHYTO TRIPPURP                                                   
1     HBO                     58.422857              57.636377   
      HBSOCREC                37.500000              15.903556   
16    HBSOCREC                42.379310              46.194205   
      NHB                     49.235294              39.987800   

                Unweighted Share of Allbike  Unweighted Share of RecBike  \
WHYTO TRIPPURP                                                             
1     HBO                          0.087675                     0.732218   
      HBSOCREC                     0.009018                     0.075314   
16    HBSOCREC                     0.014529                     0.121339   
      NHB                          0.008517                     0.071130   

                    WTTRDFIN  Weighted Share of Allbike  \
WHYTO TRIPPURP                                            
1     HBO       8.943763e+07                   0.054821   
      HBSOCREC  7.181806e+06                   0.004402   
16    HBSOCREC  1.279703e+07                   0.007844   
      NHB       4.743096e+06                   0.002907   

                Weighted Share of RecBike  
WHYTO TRIPPURP                             
1     HBO                        0.783444  
      HBSOCREC                   0.062910  
16    HBSOCREC                   0.112098  
      NHB                        0.041548

In [ ]:
with pd.ExcelWriter(os.path.join(output_location, 'NHTS_2017_bike_trips.xlsx')) as writer:
    workbook = writer.book
    worksheet = workbook.add_worksheet('readme')
    worksheet.write('A1', str(datetime.datetime.now())) 
    worksheet.write('A2', 'This file contains the analysis of bike trips in NHTS 2017. The bike trips are filtered by the following criteria: ')
    worksheet.write('A3', '1. Bike mode')
    worksheet.write('A4', f'2. Travelled in months: {GOODMONTH_FOR_BIKE}')
    worksheet.write('A5', f"3. Travelled on {config['WEEKDAYS']}")
    worksheet.write('A6', '4. Distance <= 200 miles and > 0')
    worksheet.write('A8', 'A round trip (LOOP_TRIP = 1) is counted as 2 trips, recorded in Num_Trips.')

    worksheet.write('A10', 'Data Folder:')
    worksheet.write('B10', output_location)
  
    bike_by_purp_table = bikes_by_purp.copy()
    bike_by_purp_table.loc['Total'] = bike_by_purp_table.select_dtypes(include = ['number']).sum()
    
    share_col = 'Share'
    bike_by_purp_table[share_col] = bike_by_purp_table[share_col].apply(lambda x: f'{x:.2%}')
    num_cols = bike_by_purp_table.select_dtypes(include=['number']).columns.difference([share_col])
    bike_by_purp_table[num_cols] = bike_by_purp_table[num_cols].round(2)
    bike_by_purp_table.to_excel(writer, sheet_name='AllBikeTrips')

    rec_bikes_by_purp_table = rec_bikes_by_purp.copy()
    rec_bikes_by_purp_table.loc['Total'] = rec_bikes_by_purp_table.select_dtypes(include = ['number']).sum()
    share_cols = rec_bikes_by_purp_table.columns[rec_bikes_by_purp_table.columns.str.contains('Share')]
    rec_bikes_by_purp_table[share_cols] = rec_bikes_by_purp_table[share_cols].apply(lambda col: col.map("{:.2%}".format))
    num_cols = rec_bikes_by_purp_table.select_dtypes(include=['number']).columns.difference(share_cols)
    rec_bikes_by_purp_table[num_cols] = rec_bikes_by_purp_table[num_cols].round(2)

    rec_bikes_by_purp_table.to_excel(writer, sheet_name='RecBikeTrips')

    worksheet = workbook.add_worksheet('recbike_summary')
    worksheet.write('A1', 'Recreational Bike Trips Summary')
    nhbased_cnt = rec_bikes_by_purp.loc[rec_bikes_by_purp.index.get_level_values(TRIP_PURP_4STEP) == 'NHB', 'Num_Trips'].sum()
    hbased_cnts = recb_unweighted_total - nhbased_cnt
    nhb_weighted_trips = rec_bikes_by_purp.loc[rec_bikes_by_purp.index.get_level_values(TRIP_PURP_4STEP) == 'NHB', TRIP_WEIGHT_COLUMN].sum()
    hbased_weighted_trips = recb_weighted_total - nhb_weighted_trips
    
    print('Calculate the average distance of rec bike trips')
    hb_total_BMT_unweighted = rec_bikes_by_purp.loc[rec_bikes_by_purp.index.get_level_values(TRIP_PURP_4STEP).isin(['HBSOCREC', 'HBO']), 'TRPMILES'].sum()
    nhb_total_BMT_unweighted = rec_bikes_by_purp.loc[rec_bikes_by_purp.index.get_level_values(TRIP_PURP_4STEP).isin(['NHB']), 'TRPMILES'].sum()
    hbased_records = rec_bikes_by_purp.loc[rec_bikes_by_purp.index.get_level_values(TRIP_PURP_4STEP).isin(['HBSOCREC', 'HBO']), 'Num_Record'].sum()
    nhbased_records = rec_bikes_by_purp.loc[rec_bikes_by_purp.index.get_level_values(TRIP_PURP_4STEP).isin(['NHB']), 'Num_Record'].sum()
    hb_avg_dist_unweighted = hb_total_BMT_unweighted / hbased_records
    nhb_avg_dist_unweighted = nhb_total_BMT_unweighted / nhbased_records

    recsumary = {
        'Count': [hbased_cnts, nhbased_cnt],
        'Weighted_trips': [hbased_weighted_trips, nhb_weighted_trips],
        'Unweighted_share_of_all_bike': [hbased_cnts/bikes_unweighted_total, nhbased_cnt/bikes_unweighted_total],
        'Weighted_share_of_all_bike': [hbased_weighted_trips/bikes_weighted_total, nhb_weighted_trips/bikes_weighted_total],
        'Avg_Distance_unweighted': [hb_avg_dist_unweighted, nhb_avg_dist_unweighted]
    }

    recsum_df = pd.DataFrame(recsumary, index=['HomeBased Rec Bike', 'Non-home Based Rec Bike'])
    share_cols = recsum_df.columns[recsum_df.columns.str.contains('share', case=False)]
    recsum_df[share_cols] = recsum_df[share_cols].apply(lambda col: col.map("{:.2%}".format))
    recsum_df.to_excel(writer, sheet_name='recbike_summary', startrow = 1)
    worksheet.write('A10', 'Total bike counts (unweighted):')
    worksheet.write('B10', bikes_unweighted_total)
    worksheet.write('A11', 'Total bike trips (weighted):')
    worksheet.write('B11', bikes_weighted_total)
    worksheet.write('A12', 'Average rec bike travel distance (unweighted):')
    worksheet.write('B12', (rec_bikes_by_purp['TRPMILES'].sum() / rec_bikes_by_purp['Num_Record'].sum()))

print('Done')

Calculate the average distance of rec bike trips
Done


In [18]:
hbased_records, hb_avg_dist_unweighted

(222, 7.870360360360362)